In [ ]:
!pip install pandas

In [ ]:
import datetime
from datetime import datetime as dt
import pandas as pd

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters
from google.cloud.bigtable.row_set import RowSet 

In [ ]:
INSTANCE_ID = 'big-table-db2'
TABLE_ID = 'plant2'

In [ ]:
client = bigtable.Client.from_service_account_json('../../lofty-psyche-328203-949244c34274.json', admin = True)

In [ ]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

In [ ]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

In [ ]:
print('Creating all family columns')

plant_cfId = 'plant'
customer_cf = table.column_family(plant_cfId)
customer_cf.create()

telemetry_cfId = 'telemetry'
items_cf = table.column_family(telemetry_cfId)
items_cf.create()

In [ ]:
df = pd.read_csv('../data/Plant_2_Weather_Sensor_Data.csv')
len(df[0:3])
value = str(df['AMBIENT_TEMPERATURE'][0]).encode()
#row_key = 'plant#{}#{}'.format(df['PLANT_ID'][0], df['DATE_TIME'][0]).encode()
#row_key

In [ ]:
print('Writing orders to the table')
rows = []
for line in range(len(df)):
    row_key = 'plant#{}#{}'.format(df['PLANT_ID'][line], df['DATE_TIME'][line]).encode()
    date_time = datetime.datetime.strptime(df['DATE_TIME'][line], '%Y-%m-%d %H:%M:%S')
    row = table.direct_row(row_key)

    row.set_cell(plant_cfId, 'source_key'.encode(), df['SOURCE_KEY'][line], timestamp = date_time)

    row.set_cell(telemetry_cfId, 'ambient_temp'.encode(), str(df['AMBIENT_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'module_temp'.encode(), str(df['MODULE_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'irradiation'.encode(), str(df['IRRADIATION'][line]), timestamp = date_time)

    rows.append(row)

table.mutate_rows(rows)

In [ ]:
print('Reading a range of rows')

row_set = RowSet()
row_set.add_row_range_from_keys(
    start_key = b'plant#4136001#2020-06-01 06:00:00',
    end_key = b'plant#4136001#2020-06-01 12:00:00'
)

col_filter = row_filters.ColumnQualifierRegexFilter('ambient_temp'.encode())

rows = table.read_rows(row_set=row_set, filter_=col_filter)

for row in rows:
    print_row(row)

In [ ]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")